In [31]:
import pandas as pd

In [32]:
data_location = 'C:/Users/kaley/OneDrive/Documents/Coding Stuff/Data Science/Mental Health Project/Sample Data'
step_data_location = data_location + '/Withings/aggregates_steps.csv'


In [33]:
step_data = pd.read_csv(step_data_location)

In [34]:
step_data['Date'] = pd.to_datetime(step_data.date)
step_data['Date'] = step_data['Date'].dt.date
step_data = step_data.rename(columns={'value':'Steps'})
step_data = step_data[['Date','Steps']]

In [35]:
step_data.head()

,Date,Steps
0,2023-07-10,11333
1,2023-07-09,7697
2,2023-07-08,6183
3,2023-07-07,8002
4,2023-07-06,5622


In [36]:
mood_data_location = data_location + '/FinchExport_2023-06-22/Mood.json'


In [37]:
mood_data = pd.read_json(mood_data_location, lines=True)

In [38]:
satisfaction_data = mood_data[mood_data['mood_type'] == 'satisfaction']

In [39]:
satisfaction_data.head()

,dt,mood_type,value,updated_time
2,"Wed, 2 Nov 2022 22:06:19",satisfaction,1,2022-11-02 22:06:19
5,"Wed, 14 Dec 2022 03:26:55",satisfaction,4,2022-12-14 03:26:55
6,"Sat, 20 May 2023 00:25:57",satisfaction,4,2023-05-20 00:25:57
8,"Sat, 3 Dec 2022 01:05:45",satisfaction,2,2022-12-03 01:05:45
27,"Sun, 1 Jan 2023 04:07:36",satisfaction,5,2023-01-01 04:07:36


In [40]:
satisfaction_data['Datetime'] = pd.to_datetime(satisfaction_data.dt)
satisfaction_data = satisfaction_data.sort_values(by=['Datetime'])

C:\Users\kaley\AppData\Local\Temp\ipykernel_16796\3646379311.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  satisfaction_data['Datetime'] = pd.to_datetime(satisfaction_data.dt)


In [41]:
df = satisfaction_data[['Datetime', 'updated_time']].assign(NE=satisfaction_data.Datetime != satisfaction_data.updated_time)

In [42]:
df.groupby('NE').count()

,Datetime,updated_time
NE,,
False,187,187


In [43]:
satisfaction_data = satisfaction_data[['Datetime','value']]

In [44]:
satisfaction_data.head()

,Datetime,value
57,2022-10-26 20:55:38,3
132,2022-10-27 22:19:43,2
650,2022-10-29 02:09:04,4
223,2022-10-29 22:44:40,3
782,2022-10-30 22:08:04,4


In [45]:
# remove time from Datetime? but also need to deal with multiple values per day sitch

In [46]:
satisfaction_data['Date'] = satisfaction_data['Datetime'].dt.date
satisfaction_data['Time'] = satisfaction_data['Datetime'].dt.time
satisfaction_data = satisfaction_data.rename(columns={'value':'Satisfaction_Score'})

In [47]:
satisfaction_data.head()

,Datetime,Satisfaction_Score,Date,Time
57,2022-10-26 20:55:38,3,2022-10-26,20:55:38
132,2022-10-27 22:19:43,2,2022-10-27,22:19:43
650,2022-10-29 02:09:04,4,2022-10-29,02:09:04
223,2022-10-29 22:44:40,3,2022-10-29,22:44:40
782,2022-10-30 22:08:04,4,2022-10-30,22:08:04


In [48]:
# kewl! lets try to join step and satisfaction data!

In [49]:
step_satisfaction_df = satisfaction_data.set_index('Date').join(step_data.set_index('Date')).reset_index().sort_values(by=['Date'])

In [50]:
# apparently some days have multiple
# score should be appiinted to date not the day that I entered the score

In [51]:
step_satisfaction_df

,Date,Datetime,Satisfaction_Score,Time,Steps
0,2022-10-26,2022-10-26 20:55:38,3,20:55:38,245
1,2022-10-27,2022-10-27 22:19:43,2,22:19:43,259
2,2022-10-29,2022-10-29 02:09:04,4,02:09:04,786
3,2022-10-29,2022-10-29 22:44:40,3,22:44:40,786
4,2022-10-30,2022-10-30 22:08:04,4,22:08:04,1267
...,...,...,...,...,...
182,2023-06-17,2023-06-17 23:29:24,5,23:29:24,9361
183,2023-06-18,2023-06-18 23:12:39,2,23:12:39,4007
184,2023-06-20,2023-06-20 00:03:48,4,00:03:48,3517
185,2023-06-20,2023-06-20 22:24:04,4,22:24:04,3517


In [52]:
# if TIME is after midnight and the previous DATE has no satisfaction value

In [53]:
# TODO: I don't like this process, seems messy, alternative method for filling missing dates/values?
# I have the values need to apply them to missing dates

In [54]:
# is there a more efficient way to do this?
min_date = step_satisfaction_df.Date.min()
max_date = step_satisfaction_df.Date.max()

idx = pd.date_range(min_date, max_date)
df = pd.DataFrame(index=idx)

In [55]:
# df.head()

In [56]:
# step_satisfaction_df = satisfaction_data.set_index('Date').join(df, how='right')

In [57]:
# step_satisfaction_df.reset_index().rename(columns={'index': 'Date', 'Datetime': 'score_entered_time'})

In [58]:
#  should I just change the duplicated date to the previous date - that is missing the score
#  need another column, for the date associated with the score, while datetime is associated with the time the score was given

In [59]:
# okay different attempt

In [60]:
step_satisfaction_df.head()

,Date,Datetime,Satisfaction_Score,Time,Steps
0,2022-10-26,2022-10-26 20:55:38,3,20:55:38,245
1,2022-10-27,2022-10-27 22:19:43,2,22:19:43,259
2,2022-10-29,2022-10-29 02:09:04,4,02:09:04,786
3,2022-10-29,2022-10-29 22:44:40,3,22:44:40,786
4,2022-10-30,2022-10-30 22:08:04,4,22:08:04,1267


In [61]:
step_satisfaction_df.rename(columns={'Datetime':'Time_score_entered'})

,Date,Time_score_entered,Satisfaction_Score,Time,Steps
0,2022-10-26,2022-10-26 20:55:38,3,20:55:38,245
1,2022-10-27,2022-10-27 22:19:43,2,22:19:43,259
2,2022-10-29,2022-10-29 02:09:04,4,02:09:04,786
3,2022-10-29,2022-10-29 22:44:40,3,22:44:40,786
4,2022-10-30,2022-10-30 22:08:04,4,22:08:04,1267
...,...,...,...,...,...
182,2023-06-17,2023-06-17 23:29:24,5,23:29:24,9361
183,2023-06-18,2023-06-18 23:12:39,2,23:12:39,4007
184,2023-06-20,2023-06-20 00:03:48,4,00:03:48,3517
185,2023-06-20,2023-06-20 22:24:04,4,22:24:04,3517


In [ ]:
 # so algorithm:
    # check for duplicates in Date col?
    # then replace date that has earlier time w one day previous
    
# THIS SHOULD BE DONE BEFORE merging w step data ? join on Date not Time_score_entered